In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

In [2]:
from src.Retriever import ThresholdRetriever, ThresholdRetrieverConfig

from src.Retriever import RerankRetriever, RerankRetrieverConfig
from src.Reader import LLM_Model, LLM_Config

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = ThresholdRetrieverConfig()

In [ ]:
retriever = ThresholdRetriever(config)

In [5]:
output = retriever.invoke("Знаменитая певица")

In [3]:
reader_config = LLM_Config()
retriever_config = RerankRetrieverConfig()

In [4]:
reader = LLM_Model(reader_config)

Loading checkpoint shards: 100%|██████████| 4/4 [03:35<00:00, 53.80s/it]


In [28]:
from typing import List

class LLM_UncertaintyScorer:
    def __init__(self, config: UncertaintyScorerConfig, reader: LLM_Model):
        self.config = config
        self.reader = reader
        self.system_prompt = self.config.system_prompt.format(
            cert_label=self.config.cert_label,uncert_label=self.config.uncert_label)

    def predict(self, query: str, contexts: List[str]):
        user_prompts = [self.config.user_prompt_template.format(q=query, c=cntx) for cntx in contexts]
        output = self.reader.generate(user_prompts, system_prompt=self.system_prompt, 
                                      gen_params=self.config.gen, batch_size=self.config.batch_size)

        predicted_scores = []
        for raw_scores in output:
            for raw_score in list(map(lambda v: v.lower(), raw_scores)):
                print(raw_score)
                cert_dist = distance(self.config.cert_label, raw_score)
                unsert_dist = distance(self.config.uncert_label, raw_score)
                predicted_scores.append(self.config.cert_label if cert_dist < unsert_dist else self.config.uncert_label)

        return predicted_scores

class RerankRetriever:
    def __init__(self, config: RerankRetrieverConfig, reader: LLM_Model):
        self.config = config
        self.stage1_retriever = ThresholdRetriever(self.config.stage1_retriever_config)
        self.scorer = LLM_UncertaintyScorer(self.config.scorer_config, reader)

    def rerank(self, query: str, docs: List[List[object]]):
        predicted_scores = self.scorer.predict(query, list(map(lambda doc: doc[1], docs)))
        relevant_docs = [doc for i, doc in enumerate(docs) if predicted_scores[i] == self.scorer.config.cert_label]
        return relevant_docs 
    
    def invoke(self, query: str):
        stage1_relevant_docs = self.stage1_retriever.invoke(query)
        stage2_relevant_docs = self.rerank(query, stage1_relevant_docs)
        return stage2_relevant_docs

In [33]:
retriever_config.scorer_config.cert_label = "1"
retriever_config.scorer_config.uncert_label = "0"

In [34]:
retriever = RerankRetriever(retriever_config, reader)

No sentence-transformers model found with name /trinity/home/team06/workspace/mikhail_workspace/rag_project/models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [37]:
output = retriever.invoke("Когда родился Александр Тургенев??")

1
0
0
0
0
0
0
1
0
0


In [38]:
output

[[0.14319467544555664,
  'Ива́н Серге́евич Турге́нев (28 октября (9 ноября) 1818, Орёл, Российская империя — 22 августа (3 сентября) 1883, Буживаль, Франция) — русский писатель-реалист, поэт, публицист, драматург, переводчик. Один из классиков русской литературы, внёсших наиболее значительный вклад в её развитие во второй половине XIX века. Член-корреспондент императорской Академии наук по разряду русского языка и словесности (1860), почётный доктор Оксфордского университета (1879), почётный член Московского университета (1880).',
  {'chunk_id': 5700535745,
   'doc_id': 5700535745,
   'next_chunk_id': 'None',
   'prev_chunk_id': 'None'}],
 [0.19038772583007812,
  'В начале 1842 года он подал в Московский университет просьбу о допуске к экзамену на степень магистра философии, однако в то время штатного профессора философии в университете не было, и его просьбу отклонили. Не устроившись в Москве, Тургенев удовлетворительно выдержал экзамен на степень магистра по греческой и латинской фил